In [ ]:
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
torch.manual_seed(1)

In [12]:
with open('training_data.pickle','rb') as f:
    input = pickle.load(f)